# Building the Hello World Skill
This notebook demonstrates how to use the Cortex Python SDK to build a simple Skill.  For simple cases, the entire Skill can be defined, tested, and deployed right from a notebook.

In [ ]:
from cortex import Cortex
%reload_ext cortex

## Building a Cortex Action using iPython Magic
The Cortex Action is a self-contained code module that encapsulates a task - in other words, it is a function in computer science terms.  It can be a short running service, long running batch job, or stateful microservice.  For our simple "Hello World" example, we can create our Action using a function and leverage the __%%cortex_action__ iPython cell magic to automatically deploy it to the Cortex Cloud.

Any function can be configured into a Cortex Action with ease. 

To help think about this, imagine the whole action as a sandwich. The bread is where Cortex comes in, but the meat is the code that would 
be the same in any other environment. 

For Cortex Actions and Skills, one must handle the inputs and outputs properly. 

The inputs are taken as JSON objects, and at the same time the outputs are returned as JSON objects.

These JSON objects can contains as many inputs or outputs as any normal function could. 

To illustrate this, here is a very basic example below using the classic 'Hello World' example:

In [ ]:
%%cortex_action --name 'class/hello-world-YOUR_INITIALS' --function hello_world

from cortex import Message

def hello_world(params):
    msg = Message(params)
    text = msg.payload.get('text')
    return Message.with_payload({'message': 'Hello %s!' % text}).to_params()

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did above for '--name'
action = cortex.action('class/hello-world')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

Invoke the Action using a Message object.  Here we just pass in the expected _text_ parameter in the Message payload.

In [ ]:
from cortex import Message
rs = action.invoke(Message.with_payload({'text': 'Cortex'}))
rs.payload

## Building a Cortex Skill from the Hello World Action
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Hello World" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
hello_world = builder.skill('class/hello-world-YOUR_INITIALS').title('Hello World-YOUR_INITIALS').description('Introductory Hello World Skill by YOUR_INITIALS')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares which Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
hello_world = hello_world.input('say-hello').title('Your Name').parameter(name='text', type='string').all_routing(action, 'greeting').build()

In the previous step, we referenced an Output called **greeting**.  We can create that Output here using the Output sub-builder.

In [ ]:
hello_world = hello_world.output('greeting').title('Greeting').parameter(name='message', type='string').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to__camel_ method.

In [ ]:
hello_world.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = hello_world.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='say-hello', message=Message.with_payload({'text': 'Cortex'}))
rs.payload

## Creating Your Own Action

Now that you have seen some basic action and skill creation, it's time to try it on your own. Below are two basic code samples, and it's your job to turn them into actions. 

# Converting the movie_maker and replace_cast functions to Cortex actions

Remember, you can import whichever libraries you need. If certain libraries require a **pip install**, they can be added into the 

docker container through the cell magic via a **'--requirements'** flag up top. 

Example:
For the 'seaborn' library, one cannot simply import it. It must be installed via pip first. The cell magic up top would then look like:

**'%%cortex_action --name 'class/my-function' --function my_function --requirements seaborn'**

Any other libraries that must be installed can be added to the requirements flag with a comma. ('--requirements seaborn,nltk')

To create a daemon or job, add a '--daemon' or '--job' flag to the top of the cell magic, nothing else needs to change. 

The same cell magic above for creating a daemon would look like

**'%%cortex_action --name 'class/my-function' --function my_function --requirements seaborn' --daemon**

**Note** For the movie_maker function, you will need to add the 'faker' library as a requirement

Using the hello_world walkthrough as an example, convert these two functions to Cortex Actions followed by Cortex Skills yourself

The movie_maker function takes in the name of a movie as a string, and outputs a dictionary with four cast members

In [ ]:
%%cortex_action --name 'class/movie_maker-YOUR_INITIALS' --function movie_maker

from faker import Faker
def movie_maker(params):
    Message=msg.params
    fake = Faker()
    movie = {'name_of_movie': name_of_movie, 'lead_1': fake.name(), 'lead_2': fake.name(), 'support_1': fake.name(), 'support_2': fake.name()}
    return movie

In [ ]:
movie_maker('Johns Big Adventure')

The replace_cast function takes in the role to replace as a string ('lead_1', 'support_2', etc.), the movie details as a dictionary (The output of the movie_maker function), and the name of the cast member you want to replace the role as a string ('Keanu Reeves', 'Tom Cruise', etc.)

In [ ]:
def replace_cast(role, movie, new_cast):
    movie[role] = new_cast
    return movie

In [ ]:
replace_cast('support_2', movie_maker('Johns Big Adventure'), 'Johnny Depp')

# Testing the movie_maker action

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did in the action header                                                                                              d above for '--name'
action = cortex.action('class/movie_maker-YOUR_INITIALS')

In [ ]:
from cortex import Message
rs = action.invoke(Message.with_payload({'name_of_movie': 'Johns Big Adventure'}))
movie_maker_output = rs.payload['movie']
#We want to keep track of this output payload for use as input when invoking the replace_cast function
movie_maker_output

# Testing the replace_cast action

Now try invoking the replace_cast action yourself!

Make sure to add your initials at the end of the action name like you did when you created it.

In [ ]:
#initialize your replace_cast action here


In [ ]:
#invoke your replace_cast action here



# Creating the movie_maker Skill

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Hello World" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
b = builder.skill('class/movie_maker-YOUR_INITIALS').title('Movie Maker-YOUR_INITIALS').description('Skill which generates a fake cast for a movie-YOUR_INITIALS')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares which Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
b = b.input('make-movie').title('Name of Movie').parameter(name='name_of_movie', type='string').all_routing(action, 'movie').build()

In the previous step, we referenced an Output called **greeting**.  We can create that Output here using the Output sub-builder.

In [ ]:
b = b.output('movie').title('Movie').parameter(name='movie', type='object', required=True).parameter(name='role', type='string', required=True).parameter(name='new_cast', type='string', required=True).build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to__camel_ method.

In [ ]:
b.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = b.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

In [ ]:
rs = skill.invoke(input_name='make-movie', message=Message.with_payload({'name_of_movie': 'Neils Big Adventure'}))
rs.payload

# Creating the replace_cast Skill

Try creating the replace_cast skill yourself! Be sure to include your initials at the end of things like the skill name and title

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Hello World" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

In [ ]:
#Skill metadata goes here


Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares which Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

**Note** For inputs with multiple parameters, at the end of one parameter **'.parameter(name='greeting', type='string)'**, 

you can simply chain the .parameter calls like **'.parameter(name='greeting', type='string').parameter(name='age', type = 'number')'**

The replace_cast function has three parameters.

In [ ]:
#Skill input data goes here


In the previous step, we referenced an Output called **greeting**.  We can create that Output here using the Output sub-builder.

In [ ]:
#Skill output data goes here


## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to__camel_ method.

In [ ]:
#Camel preview goes here


## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
#Calling the build function on our skill goes here


**Note** We want to invoke this skill with the output of the movie maker action. We saved the payload data from that action invocation earlier, we can use it now for testing out the replace_cast skill

In [ ]:
#Invoking the skill goes here
